In [6]:
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import requests
session = requests.session()
import re

In [2]:
def tokenization(feedback, flag): #токенизация, очищение от знаков препинания и лемматизация
  liste = []
  for i in feedback.split():
    i = i.strip(punctuation)
    if i.isalpha():
      if flag == True:
        i = morph.parse(i.lower())[0].normal_form
      liste.append(i.lower())
  return liste

In [3]:
def tok_words(word):
  if isinstance(word, str) == True:
    norm = morph.parse(word.lower())[0].normal_form
  else:
    word = word.split()
    if len(word) == 2:
      if morph.parse(word[0].lower())[0].tag.POS == 'NOUN':
        x = morph.parse(word[0].lower())[0].tag.gender
        word[1] = morph.parse(word[1].lower())[0].inflect({x}).word
      elif morph.parse(word[1].lower())[0].tag.POS == 'NOUN':
        x = morph.parse(word[1].lower())[0].tag.gender
        word[0] = morph.parse(word[0].lower())[0].inflect({x}).word
      return ' '.join(word)
    else:
      k = []
      for i in word:
        k.append(morph.parse(word[i].lower())[0].normal_form)
      norm = ' '.join
  return norm

In [4]:
def pos(mot):
  for i in range(len(mot)):
    mot[i] = str(morph.parse(mot[i])[0].tag.POS)
  return ' '.join(mot)

In [19]:
def pos_word(word, untouched):
  for i in range(len(word)):
    if word[i] not in untouched:
      word[i] = str(morph.parse(word[i])[0].tag.POS)
  return ' '.join(word)

In [7]:
title_phrases = dict()
for n in range(0, 101, 50):
  link = "https://proza.ru/avtor/adacva1&s=" + str(n)
  soup = BeautifulSoup(session.get(link).text)
  refer = soup.find_all('a', {'class':"poemlink"})
  list_of_phrases = []
  for i in refer:
    story = 'https://proza.ru' + i['href']
    soup1 = BeautifulSoup(session.get(story).text)
    text = soup1.find('div', {'class':'text'})
    title = soup1.find('h1')
    title_phrases[title.text] = text.text
    pattern = re.compile(r'([А-ЯA-Z]((т.п.|т.д.|пр.|г.)|[^?!.\(]|\([^\)]*\))*[.?!])')
    for j, sent in enumerate(pattern.findall(title_phrases[title.text])):
        list_of_phrases.append(sent[0])
    title_phrases[title.text] = list_of_phrases.copy()
    list_of_phrases.clear()

In [22]:
def search(sequence):
  count = 0
  parts_os = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS',
      'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ',
      'PRCL', 'INTJ']
  if sequence[0] == '"':
    fla = False
  else:
    fla = True
  pos_search = False
  mix = False
  untouched = []
  for i in sequence.split():
    if i in parts_os:
      pos_search = True
    else:
      mix = True
      untouched.append(i)
  for i in title_phrases.items():
    k = i[1]
    for j in k:
      if pos_search == True and mix == False:
        pdd = pos(tokenization(j, flag=False))
        pdd_res = re.search(r'\b' + re.escape(sequence) + r'\b', pdd)
        if pdd_res:
          print(j, "[Из произведения", '"' + i[0] + '"]')
          count += 1
      elif pos_search == True and mix == True:
        pdd = pos_word(tokenization(j, flag=False), untouched)
        pdd_res = re.search(r'\b' + re.escape(sequence) + r'\b', pdd)
        if pdd_res:
          print(j, "[Из произведения", '"' + i[0] + '"]')
          count += 1
      else:
        if fla == True:
          p = ' '.join(tokenization(j, flag=True))
          p2 = ' '.join(tokenization(j, flag=False))
          tok = tok_words(sequence.strip('"'))
          res = re.search(r'\b' + re.escape(tok) + r'\b', p)
          res1 = re.search(r'\b' + re.escape(sequence.strip('"')) + r'\b', p2)
          if res or res1:
            print(j, "[Из произведения", '"' + i[0] + '"]')
            count += 1
        elif fla == False:
          p2 = ' '.join(tokenization(j, flag=False))
          res1 = re.search(r'\b' + re.escape(sequence.strip('"')) + r'\b', p2)
          if res1:
            print(j, "[Из произведения", '"' + i[0] + '"]')
            count += 1
  return 'По запросу ' + sequence + ' найдено ' + str(count) + ' результатов'

In [31]:
print(search('очень ADJF'))

Конечно, несколько эмоциональной была она, но эмоциями могла управлять, особенно перед не очень близкими людьми. [Из произведения "Мир шире кухни"]
Но у входа в подъезд стояли Мария и Зинаида, они кормили голубей, оголодавших за холодную зиму и не очень теплую весну. [Из произведения "Наследство. Деньги - больше денег"]
Очень удобное место в кухне нашла Светлана для телевизора. [Из произведения "Фильм с борщом"]
Тапочки у нее были даже очень удобные. [Из произведения "Фильм с борщом"]
Уж очень аккуратной была у мужчины стрижка, - оценила прическу Эдика Марфа. [Из произведения "Под цветным платком"]
Давно-давно работал он в каком-то научном институте, очень секретном и закрытом. [Из произведения "Вальс на отдыхе"]
Вот работаю, получу деньги за работу и отнесу в организацию очень хорошую, - начала беседу на интересную для нее тему. [Из произведения "Будет новая квартира"]
На пороге стояла уж очень молодая женщина, в ярком, но строгом красном костюме. [Из произведения "Незнакомка в коллек